In [1]:
import pandas as pd
import os

In [2]:
events=pd.read_json("../../simulation.jsonlines", orient="records",lines=True)
planes=pd.read_json("../../plans.jsonlines", orient="records",lines=True)

In [3]:
planes

,items,simulationId,trucks
0,"[{'itemId': '0', 'locationId': '326ae8dc7810de...",a2bfd344-3b14-462a-82e7-d42aca54a650,"[{'items': ['23', '24', '4', '25', '26', '27',..."
1,"[{'itemId': '0', 'locationId': '271c4e156a04e8...",a8a11526-6b9b-4143-bfb1-ffcae600a17e,"[{'items': ['62', '63', '69', '70', '71', '21'..."
2,"[{'itemId': '0', 'locationId': '6aa95e84c67637...",2f956e3a-25df-4c5e-bb50-92d92dc9ba00,"[{'items': ['79', '20', '21', '5', '76', '59',..."
3,"[{'itemId': '0', 'locationId': '0078f3bf6db092...",45c9a42a-7cb5-4266-b0aa-a05b3f446390,"[{'items': ['3', '4', '5', '6', '72', '11', '1..."
4,"[{'itemId': '0', 'locationId': '92e059821c9ca8...",864bd9f3-074d-45e8-a032-5a01237ae290,"[{'items': ['77', '78', '79', '16', '17', '91'..."
...,...,...,...
94,"[{'itemId': '0', 'locationId': '281ec8ac82cccd...",81acb266-ab82-497b-92d9-fae490cc4823,"[{'items': ['36', '37', '49', '6', '7', '8', '..."
95,"[{'itemId': '0', 'locationId': '950001b613cb58...",f007019f-866d-48be-a0cb-bd96328ee43b,"[{'items': ['46', '47', '48', '49', '40', '24'..."
96,"[{'itemId': '0', 'locationId': 'b40599012e4fba...",f0f90050-5a0f-4965-864e-d6bc0d02b5a0,"[{'items': ['16', '17', '42', '43', '44', '45'..."
97,"[{'itemId': '0', 'locationId': '074dc13529fd8b...",e33505ca-bfdb-496f-bcfd-b8b913a42637,"[{'items': ['62', '63', '64', '86', '10', '11'..."


## Tiempos de entrega

In [159]:
entregas = events[events.eventType.isin(["Truck ended delivering", "Truck started delivering"])]
entregas = entregas.sort_values("eventType",ascending=True)
entregas = entregas.sort_values(by=["simulationId", "truckId", "eventTime"])

In [160]:
entregas["deliverTime"] = entregas["eventTime"].diff()
entregas = entregas[entregas.eventType == "Truck ended delivering"]
entregas

,eventDescription,eventTime,eventType,simulationId,truckId,deliverTime
13021,"Packet 78 to (calle del Poeta Blas de Otero, 2...",872250,Truck ended delivering,007f3d08-14a7-4a93-af9e-f0d9cfbcea94,3321FBL,120630.0
13112,"Packet 79 to (calle del Poeta Blas de Otero, 2...",992820,Truck ended delivering,007f3d08-14a7-4a93-af9e-f0d9cfbcea94,3321FBL,120570.0
13566,"Packet 50 to (calle de Valdecanillas, 59, Madr...",1536120,Truck ended delivering,007f3d08-14a7-4a93-af9e-f0d9cfbcea94,3321FBL,60600.0
13614,"Packet 51 to (calle de Valdecanillas, 59, Madr...",1596720,Truck ended delivering,007f3d08-14a7-4a93-af9e-f0d9cfbcea94,3321FBL,60600.0
13666,"Packet 52 to (calle de Valdecanillas, 59, Madr...",1657320,Truck ended delivering,007f3d08-14a7-4a93-af9e-f0d9cfbcea94,3321FBL,60600.0
...,...,...,...,...,...,...
11113,"Packet 42 to (calle de la Comunidad de Madrid,...",7565580,Truck ended delivering,fea13535-ade6-4215-96ca-dab5b4ef309b,6965XLY,60600.0
11176,"Packet 43 to (calle de la Comunidad de Madrid,...",7626180,Truck ended delivering,fea13535-ade6-4215-96ca-dab5b4ef309b,6965XLY,60570.0
11655,"Packet 79 to (calle San Andrés, 21, Majadahond...",8243580,Truck ended delivering,fea13535-ade6-4215-96ca-dab5b4ef309b,6965XLY,60600.0
11729,"Packet 80 to (calle San Andrés, 21, Majadahond...",8304180,Truck ended delivering,fea13535-ade6-4215-96ca-dab5b4ef309b,6965XLY,60600.0


In [4]:
events["eventType"].value_counts()

eventType
Truck started delivering    8567
Truck ended delivering      8567
Truck departed              3444
Truck arrived               3444
Truck initialized            640
Truck received packets       640
Truck departed to depot      640
Truck ended route            640
Name: count, dtype: int64

## Desanidar los planes

In [5]:
planes

,items,simulationId,trucks
0,"[{'itemId': '0', 'locationId': '326ae8dc7810de...",a2bfd344-3b14-462a-82e7-d42aca54a650,"[{'items': ['23', '24', '4', '25', '26', '27',..."
1,"[{'itemId': '0', 'locationId': '271c4e156a04e8...",a8a11526-6b9b-4143-bfb1-ffcae600a17e,"[{'items': ['62', '63', '69', '70', '71', '21'..."
2,"[{'itemId': '0', 'locationId': '6aa95e84c67637...",2f956e3a-25df-4c5e-bb50-92d92dc9ba00,"[{'items': ['79', '20', '21', '5', '76', '59',..."
3,"[{'itemId': '0', 'locationId': '0078f3bf6db092...",45c9a42a-7cb5-4266-b0aa-a05b3f446390,"[{'items': ['3', '4', '5', '6', '72', '11', '1..."
4,"[{'itemId': '0', 'locationId': '92e059821c9ca8...",864bd9f3-074d-45e8-a032-5a01237ae290,"[{'items': ['77', '78', '79', '16', '17', '91'..."
...,...,...,...
94,"[{'itemId': '0', 'locationId': '281ec8ac82cccd...",81acb266-ab82-497b-92d9-fae490cc4823,"[{'items': ['36', '37', '49', '6', '7', '8', '..."
95,"[{'itemId': '0', 'locationId': '950001b613cb58...",f007019f-866d-48be-a0cb-bd96328ee43b,"[{'items': ['46', '47', '48', '49', '40', '24'..."
96,"[{'itemId': '0', 'locationId': 'b40599012e4fba...",f0f90050-5a0f-4965-864e-d6bc0d02b5a0,"[{'items': ['16', '17', '42', '43', '44', '45'..."
97,"[{'itemId': '0', 'locationId': '074dc13529fd8b...",e33505ca-bfdb-496f-bcfd-b8b913a42637,"[{'items': ['62', '63', '64', '86', '10', '11'..."


In [6]:
pd.DataFrame(planes.trucks[0])

,items,route,truck_id
0,"[23, 24, 4, 25, 26, 27, 28, 40, 41, 42, 43, 13...","[{'destination': 'af3ea5c0e98151a51bd39c64', '...",5534TPH
1,"[74, 75, 76, 67, 68, 69, 70, 92, 93, 94, 55, 5...","[{'destination': 'faf06210580833f8c7963506', '...",2423VRT
2,"[18, 19, 57, 58, 88, 89, 90, 91, 54, 20, 21, 2...","[{'destination': 'bef2df0471ee1c26b4ab6ba9', '...",0265TYL
3,"[87, 0, 1, 2, 3, 33, 95, 96, 97]","[{'destination': '7a249403ad22a72495769f75', '...",8809GPH
4,"[31, 32, 37, 38, 39, 66]","[{'destination': '1e38734ff239ccf75bf04b95', '...",1161GJN
5,[86],"[{'destination': '7d1544d169eeeb1d917e9f02', '...",1669HSZ
6,"[77, 78, 79, 52, 53, 29, 30, 80, 81, 82, 34, 3...","[{'destination': '166c008321c564cf6c885cb2', '...",3321FBL


In [7]:
pd.DataFrame(pd.DataFrame(planes.trucks[0]).route[0])

,destination,duration,origin
0,af3ea5c0e98151a51bd39c64,197,ce6feed06bc1a9e5aff95f04
1,c0430d1e19c34a06e343a270,733,af3ea5c0e98151a51bd39c64
2,b31f6bb5c55435cb5495a62b,380,c0430d1e19c34a06e343a270
3,58a50ddf91095c052e330fd6,612,b31f6bb5c55435cb5495a62b
4,910cd4ad38bc68ff7abdd941,850,58a50ddf91095c052e330fd6
5,49a09d201cc830dc1c4e87fc,241,910cd4ad38bc68ff7abdd941
6,324beca4107779629830c6d3,689,49a09d201cc830dc1c4e87fc
7,123039c250dd5a3e7c3e4fc8,187,324beca4107779629830c6d3
8,5a13a4517d5fc3018de2dbea,1203,123039c250dd5a3e7c3e4fc8
9,ce6feed06bc1a9e5aff95f04,99,5a13a4517d5fc3018de2dbea


In [8]:
planes_desanidado = planes.join(planes.trucks.explode().apply(pd.Series), lsuffix='_planes').reset_index(drop=True)
planes_desanidado

,items_planes,simulationId,trucks,items,route,truck_id
0,"[{'itemId': '0', 'locationId': '326ae8dc7810de...",a2bfd344-3b14-462a-82e7-d42aca54a650,"[{'items': ['23', '24', '4', '25', '26', '27',...","[23, 24, 4, 25, 26, 27, 28, 40, 41, 42, 43, 13...","[{'destination': 'af3ea5c0e98151a51bd39c64', '...",5534TPH
1,"[{'itemId': '0', 'locationId': '326ae8dc7810de...",a2bfd344-3b14-462a-82e7-d42aca54a650,"[{'items': ['23', '24', '4', '25', '26', '27',...","[74, 75, 76, 67, 68, 69, 70, 92, 93, 94, 55, 5...","[{'destination': 'faf06210580833f8c7963506', '...",2423VRT
2,"[{'itemId': '0', 'locationId': '326ae8dc7810de...",a2bfd344-3b14-462a-82e7-d42aca54a650,"[{'items': ['23', '24', '4', '25', '26', '27',...","[18, 19, 57, 58, 88, 89, 90, 91, 54, 20, 21, 2...","[{'destination': 'bef2df0471ee1c26b4ab6ba9', '...",0265TYL
3,"[{'itemId': '0', 'locationId': '326ae8dc7810de...",a2bfd344-3b14-462a-82e7-d42aca54a650,"[{'items': ['23', '24', '4', '25', '26', '27',...","[87, 0, 1, 2, 3, 33, 95, 96, 97]","[{'destination': '7a249403ad22a72495769f75', '...",8809GPH
4,"[{'itemId': '0', 'locationId': '326ae8dc7810de...",a2bfd344-3b14-462a-82e7-d42aca54a650,"[{'items': ['23', '24', '4', '25', '26', '27',...","[31, 32, 37, 38, 39, 66]","[{'destination': '1e38734ff239ccf75bf04b95', '...",1161GJN
...,...,...,...,...,...,...
635,"[{'itemId': '0', 'locationId': 'd1214b27823e6d...",6deec7b7-6c11-418c-a1bd-f99a9e3261b6,"[{'items': ['53', '54', '55', '83', '84', '85'...","[53, 54, 55, 83, 84, 85, 86, 26, 27, 29, 30, 3...","[{'destination': '225cd814768e8ca390983313', '...",5534TPH
636,"[{'itemId': '0', 'locationId': 'd1214b27823e6d...",6deec7b7-6c11-418c-a1bd-f99a9e3261b6,"[{'items': ['53', '54', '55', '83', '84', '85'...","[72, 73, 74, 88, 89, 90, 91, 43, 62, 63, 64, 3...","[{'destination': '417bf3aae83127cd905f3bc9', '...",3953RLD
637,"[{'itemId': '0', 'locationId': 'd1214b27823e6d...",6deec7b7-6c11-418c-a1bd-f99a9e3261b6,"[{'items': ['53', '54', '55', '83', '84', '85'...","[6, 7, 8, 9]","[{'destination': '1e1251e186dd507f63e4d1e6', '...",5030LXK
638,"[{'itemId': '0', 'locationId': 'd1214b27823e6d...",6deec7b7-6c11-418c-a1bd-f99a9e3261b6,"[{'items': ['53', '54', '55', '83', '84', '85'...","[44, 45, 46, 47, 94, 20, 21, 22, 95, 96, 97, 6...","[{'destination': '6a33d0bcd3624ccb86febf09', '...",3321FBL


In [9]:
planes_desanidado_x2 = planes_desanidado.join(planes_desanidado.route.explode().apply(pd.Series)).reset_index(drop=True)
planes_desanidado_x2[['simulationId', 'truck_id', 'origin', 'destination', 'duration']]

,simulationId,truck_id,origin,destination,duration
0,a2bfd344-3b14-462a-82e7-d42aca54a650,5534TPH,ce6feed06bc1a9e5aff95f04,af3ea5c0e98151a51bd39c64,197
1,a2bfd344-3b14-462a-82e7-d42aca54a650,5534TPH,af3ea5c0e98151a51bd39c64,c0430d1e19c34a06e343a270,733
2,a2bfd344-3b14-462a-82e7-d42aca54a650,5534TPH,c0430d1e19c34a06e343a270,b31f6bb5c55435cb5495a62b,380
3,a2bfd344-3b14-462a-82e7-d42aca54a650,5534TPH,b31f6bb5c55435cb5495a62b,58a50ddf91095c052e330fd6,612
4,a2bfd344-3b14-462a-82e7-d42aca54a650,5534TPH,58a50ddf91095c052e330fd6,910cd4ad38bc68ff7abdd941,850
...,...,...,...,...,...
4079,6deec7b7-6c11-418c-a1bd-f99a9e3261b6,6270NFM,20ee036e147b4a1bd5af0e1c,5bec37aee4796cc965e71c88,1173
4080,6deec7b7-6c11-418c-a1bd-f99a9e3261b6,6270NFM,5bec37aee4796cc965e71c88,1f183bd64aa5bfae47843de5,1198
4081,6deec7b7-6c11-418c-a1bd-f99a9e3261b6,6270NFM,1f183bd64aa5bfae47843de5,4551fdc303a5ddafa1dcf28c,1133
4082,6deec7b7-6c11-418c-a1bd-f99a9e3261b6,6270NFM,4551fdc303a5ddafa1dcf28c,2884f6b369dc20f3051a216d,804


In [10]:
planes_desanidado_x2.sort_values(by=["simulationId", "truck_id"], inplace=True)